Tokenized text: ['the', 'cat', '[MASK]', 'on', 'the', 'mat', '.']
Input IDs: [1996, 4937, 103, 2006, 1996, 13523, 1012]
Predicted token IDs: [1012, 1012, 1012, 1012, 1012, 1012, 1012]
Predicted tokens: ['.', '.', '.', '.', '.', '.', '.']


In [ ]:
import torch
# from torch import  Tensor
from torch.nn import attention
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForMaskedLM, AdamW

In [ ]:
# Create dataset and dataloader
data = [
        "The quick brown fox jumps over the lazy dog.",
        "She sells seashells by the seashore.",
        "How much wood would a woodchuck chuck if a woodchuck could chuck wood?",
        "The quick brown fox jumps over the lazy dog.",
        "She sells seashells by the seashore.",
        "How much wood would a woodchuck chuck if a woodchuck could chuck wood?",
        "The cat in the hat",
        "To be or not to be, that is the question.",
        "A journey of a thousand miles begins with a single step.",
        "All's well that ends well.",
        "Actions speak louder than words.",
        "Beauty is in the eye of the beholder.",
]

In [ ]:
# Define a custom dataset for masked language model training

def fit_x(data, max_length=128, pad_token_id=0):
    if len(data) < max_length:
        data=data+[pad_token_id]*(max_length-len(data))
    else:
        data=data[:max_length]
    return data


def pad_data(data, pad_token_id=0, max_length=128):
    input_ids = fit_x(data["input_ids"], pad_token_id=pad_token_id, max_length=128)
    token_type_ids = fit_x(data["token_type_ids"], pad_token_id=pad_token_id, max_length=128)
    attention_mask = fit_x(data["attention_mask"], pad_token_id=pad_token_id, max_length=128)

    input_ids=torch.tensor([input_ids])
    token_type_ids=torch.tensor([token_type_ids])
    attention_mask=torch.tensor([attention_mask])

    return input_ids, token_type_ids, attention_mask



    return input_ids, token_type_ids, attention_mask

class MaskedLMDataset(Dataset):
    MAX_LEN = 128

    def __init__(self, tokenizer, data):
        self.tokenizer = tokenizer
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]
        # inputs = self.tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt', truncation=True, max_length=128, padding=True)
        inputs = self.tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=MaskedLMDataset.MAX_LEN, padding=True)

        l = len(inputs["input_ids"])
        # This is for padding
        input_ids, token_type_ids, attention_mask = pad_data(inputs, pad_token_id=self.tokenizer.pad_token_id, max_length=MaskedLMDataset.MAX_LEN)

        labels = input_ids.clone()

        # attention_mask = torch.tensor([[int(token_id != self.tokenizer.pad_token_id) for token_id in inputs["input_ids"]]])
        # Mask a token randomly
        # mask_idx = torch.randint(1, input_ids.size(1), (1,)).item()

        # mask_idx = torch.randint(1, l, (1,)).item()
        # input_ids[0, mask_idx] = tokenizer.mask_token_id

        labels = input_ids.clone()
        # labels[0, mask_idx] = input_ids[0, mask_idx]

        return input_ids.squeeze(), attention_mask.squeeze(), labels.squeeze()

In [ ]:
# Load pre-trained tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

In [2]:
# print(model.config.pad_token_id)
# print(tokenizer.pad_token_id)

0
0


In [ ]:
# prepare and training
dataset = MaskedLMDataset(tokenizer, data[:5])
dataloader = DataLoader(dataset, batch_size=30, shuffle=True)

In [ ]:
# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)

In [3]:
# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    for input_ids, attention_mask, labels in dataloader:
        optimizer.zero_grad()
        # print(labels)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/50, Loss: 4.108638286590576
Epoch 2/50, Loss: 3.137907028198242
Epoch 3/50, Loss: 2.447856903076172
Epoch 4/50, Loss: 1.9152977466583252
Epoch 5/50, Loss: 1.5211859941482544
Epoch 6/50, Loss: 1.2373405694961548
Epoch 7/50, Loss: 1.027313470840454
Epoch 8/50, Loss: 0.8672327995300293
Epoch 9/50, Loss: 0.7426521182060242
Epoch 10/50, Loss: 0.6411861777305603
Epoch 11/50, Loss: 0.5543447136878967
Epoch 12/50, Loss: 0.47995203733444214
Epoch 13/50, Loss: 0.4172589182853699
Epoch 14/50, Loss: 0.3645837604999542
Epoch 15/50, Loss: 0.3201811909675598
Epoch 16/50, Loss: 0.282680481672287
Epoch 17/50, Loss: 0.25070828199386597
Epoch 18/50, Loss: 0.22313503921031952
Epoch 19/50, Loss: 0.19927005469799042
Epoch 20/50, Loss: 0.17863687872886658
Epoch 21/50, Loss: 0.1607767641544342
Epoch 22/50, Loss: 0.1452450007200241
Epoch 23/50, Loss: 0.13167239725589752
Epoch 24/50, Loss: 0.11978603899478912
Epoch 25/50, Loss: 0.1093749850988388
Epoch 26/50, Loss: 0.10024954378604889
Epoch 27/50, Loss:

In [4]:
# Set model to evaluation mode
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [5]:
mask = "[MASK]"
# # Example text with a masked token
text = "The quick brown dog jumps over the lazy dog."


MAX_LEN = 128
# inputs = self.tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt', truncation=True, max_length=MAX_LEN, padding=True)
# inputs = tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=MAX_LEN, padding=True)

# # This is for padding
# l = len(inputs["input_ids"])
# if l < MAX_LEN:
#     inputs["input_ids"]=inputs["input_ids"]+[tokenizer.pad_token_id]*(MAX_LEN-l)
# else:
#     inputs["input_ids"]=inputs["input_ids"][:MAX_LEN]

# input_ids = torch.tensor([inputs["input_ids"]])
# # labels = input_ids.clone()
# attention_mask = torch.tensor([[int(token_id != tokenizer.pad_token_id) for token_id in inputs["input_ids"]]])

# Mask a token randomly
# mask_idx = torch.randint(1, input_ids.size(1), (1,)).item()
# mask_idx = torch.randint(1, l, (1,)).item()
# input_ids[0, mask_idx] = tokenizer.mask_token_id
# labels = input_ids.clone()
# labels[0, mask_idx] = input_ids[0, mask_idx]








# text = "I like [MASK] eat apples."
words = text.split()
times = len(words)
print(text, times)
for i in range(times):
    words = text.split()
    words[i] = mask
    seq = " ".join(words)
    print(seq)
    # Tokenize the text
    # inputs = tokenizer.encode_plus(seq, return_tensors="pt")
    inputs = tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=MAX_LEN, padding=True)

    # This is for padding
    l = len(inputs["input_ids"])
    if l < MAX_LEN:
        inputs["input_ids"]=inputs["input_ids"]+[tokenizer.pad_token_id]*(MAX_LEN-l)
    else:
        inputs["input_ids"]=inputs["input_ids"][:MAX_LEN]

    input_ids = torch.tensor([inputs["input_ids"]])
    # labels = input_ids.clone()
    attention_mask = torch.tensor([[int(token_id != tokenizer.pad_token_id) for token_id in inputs["input_ids"]]])


    # Generate predictions
    with torch.no_grad():
        outputs = model(input_ids.squeeze(), attention_mask=attention_mask.squeeze())
        predictions = outputs.logits.argmax(dim=-1)

    # Decode the predicted token
    predicted_token = tokenizer.decode(predictions[0, inputs["input_ids"].squeeze().tolist().index(tokenizer.mask_token_id)])

    print("Predicted token:", predicted_token)

The quick brown dog jumps over the lazy dog. 9
[MASK] quick brown dog jumps over the lazy dog.
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[173], line 62
     60 # Generate predictions
     61 with torch.no_grad():
---> 62     outputs = model(input_ids.squeeze(), attention_mask=attention_mask.squeeze())
     63     predictions = outputs.logits.argmax(dim=-1)
     65 # Decode the predicted token

File ~/.local/lib/python3.12/site-packages/torch/nn/modules/module.py:1532, in Module._wrapped_call_impl(self, *args, **kwargs)
   1530     return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]
   1531 else:
-> 1532     return self._call_impl(*args, **kwargs)

File ~/.local/lib/python3.12/site-packages/torch/nn/modules/module.py:1541, in Module._call_impl(self, *args, **kwargs)
   1536 # If we don't have any hooks, we want to skip the rest of the logic in
   1537 #

In [6]:
sentence = "How much wood would a woodchuck chuck if a [MASK] could chuck wood?"
# inputs = tokenizer(sentence, return_tensors="pt")

inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, truncation=True, max_length=MaskedLMDataset.MAX_LEN, padding=True)

input_ids, token_type_ids, attention_mask = pad_data(inputs, pad_token_id=tokenizer.pad_token_id, max_length=MaskedLMDataset.MAX_LEN)

mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1]
outputs = model(input_ids.squeeze(),token_type_ids=token_type_ids.squeeze(), attention_mask=attention_mask.squeeze())
predictions = outputs.logits[0, mask_idx].argmax(dim=1)
predicted_token = tokenizer.decode(predictions)
print("Predicted token:", predicted_token)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[86], line 9
      6 input_ids, token_type_ids, attention_mask = pad_data(inputs, pad_token_id=tokenizer.pad_token_id, max_length=MaskedLMDataset.MAX_LEN)
      8 mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1]
----> 9 outputs = model(input_ids.squeeze(),token_type_ids=token_type_ids.squeeze(), attention_mask=attention_mask.squeeze())
     10 predictions = outputs.logits[0, mask_idx].argmax(dim=1)
     11 predicted_token = tokenizer.decode(predictions)

File ~/.local/lib/python3.12/site-packages/torch/nn/modules/module.py:1532, in Module._wrapped_call_impl(self, *args, **kwargs)
   1530     return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]
   1531 else:
-> 1532     return self._call_impl(*args, **kwargs)

File ~/.local/lib/python3.12/site-packages/torch/nn/modules/module.py:1541, in Module._ca

In [7]:
sentence = "How much wood would a woodchuck chuck if a [MASK] could chuck wood?"
inputs = tokenizer(sentence, return_tensors="pt")
mask_idx = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
outputs = model(**inputs)
predictions = outputs.logits[0, mask_idx].argmax(dim=1)
predicted_token = tokenizer.decode(predictions)
print("Predicted token:", predicted_token)

Predicted token: [PAD]


In [8]:
text = "The [MASK] brown dog jumps over the lazy dog."
inputs = tokenizer.encode_plus(text, return_tensors="pt")

# Generate predictions
with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(-1)

# print(predictions[0].shape)
# Decode the predicted token
predicted_token = tokenizer.decode(predictions[0, inputs["input_ids"].squeeze().tolist().index(tokenizer.mask_token_id)])

print("Predicted token:", predicted_token)

Predicted token: l a z y


This is for text Generate, text completion and masked token prediction

In [9]:
prompt = "The quick brown fox"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100, num_return_sequences=1)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text:", generated_text)


text = "She sells seashells by the seashore. "
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100, num_return_sequences=1)
completed_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Completed text:", completed_text)

sentence = "How much wood would a woodchuck chuck if a [MASK] could chuck wood?"
inputs = tokenizer(sentence, return_tensors="pt")
mask_idx = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
outputs = model(**inputs)
predictions = outputs.logits[0, mask_idx].argmax(dim=1)
predicted_token = tokenizer.decode(predictions)
print("Predicted token:", predicted_token)

Generated text: the quick brown fox the fox. hunt mr. hunt the fox the fox mr. mr..... the fox the mr...... mr...
Completed text: she sells seashells by the seashore.....................................................................................
Predicted token: man


In [10]:
# Example of data augmentation
# from nlpaug.util import Action
# import nlpaug.augmenter.word as naw

# # Original data
# original_data = ["The quick brown fox jumps over the lazy dog.",
#                  "She sells seashells by the seashore.",
#                  "How much wood would a woodchuck chuck if a woodchuck could chuck wood?"]

# # Augmenter for synonym replacement
# aug = naw.SynonymAug(aug_src='wordnet')

# # Augment data
# augmented_data = []
# for sentence in original_data:
#     augmented_sentence = aug.augment(sentence)
#     augmented_data.append(augmented_sentence)

# # Print augmented data
# for idx, (original, augmented) in enumerate(zip(original_data, augmented_data)):
#     print(f"Original {idx + 1}: {original}")
#     print(f"Augmented {idx + 1}: {augmented}")
#     print()

Original 1: The quick brown fox jumps over the lazy dog.
Augmented 1: ['The ready john brown fox jumps ended the lazy dog.']

Original 2: She sells seashells by the seashore.
Augmented 2: ['She sell seashells by the coast.']

Original 3: How much wood would a woodchuck chuck if a woodchuck could chuck wood?
Augmented 3: ['How much wood would a woodchuck chuck if a marmota monax could regorge wood?']



Predicted token: t o
